In [1]:
import numpy as np
import datetime, time
from multiprocess import Process, Queue, cpu_count
import multiprocess as multip
import os, re, glob
import _pickle as cPickle
from PyQt5 import QtCore

import LaueTools.dict_LaueTools as dictLT
import LaueTools.IOLaueTools as IOLT
import LaueTools.generaltools as GT
import LaueTools.CrystalParameters as CP
import LaueTools.lauecore as LT
import LaueTools.LaueGeometry as Lgeo
import LaueTools.readmccd as RMCCD
import LaueTools.FitOrient as FitO
import LaueTools.findorient as FindO
from fast_histogram import histogram1d
import networkx as nx
import h5py
import time
import ipywidgets as widgets

tensorflow_keras = True
try:
    from keras.models import model_from_json
except:
    tensorflow_keras = False

from utils_lauenn import Symmetry, Lattice, SGLattice, chunker_list,\
                            rot_mat_to_euler, global_plots, save_sst

cpu_count_user = 8#cpu_count()
ncpu = cpu_count_user
total_progress = 0
#timermp1212 = QtCore.QTimer()
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
max_progress = 1

print("Number of CPUs available : ", ncpu)

-- OK! You are using python 3
Missing library libtiff, Please install: pylibtiff if you need open some tiff images
-- warning: module Image or PIL is not installed, but only used for templateimagematching
Number of CPUs available :  8


In [2]:
def worker(inputs_queue, outputs_queue, proc_id, run_flag):
    print(f'Initializing worker {proc_id}')
    while True:
        if not run_flag.value:
            break
        time.sleep(0.01)
        if not inputs_queue.empty(): 
            message = inputs_queue.get()
            if message == 'STOP':
                print(f'[{proc_id}] stopping')
                break
            num1, num2, meta = message
            files_worked = []

            #for ijk in trange(len(num1), desc='worker '+str(proc_id)):
            for ijk in range(len(num1)):
                
                if ijk in files_worked:
                    continue                       
                if not run_flag.value:
                    num1, files_worked = [], []
                    print(f'[{proc_id}] stopping')
                    break
                
                files, cnt, rotation_matrix, strain_matrix, strain_matrixs,\
                col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,mat_global,\
                check,detectorparameters,pixelsize,angbins,\
                classhkl, hkl_all_class0, hkl_all_class1, emin, emax,\
                material_, material1_, symmetry, symmetry1,lim_x,lim_y,\
                strain_calculation, ind_mat, ind_mat1,\
                model_direc, tolerance , tolerance1,\
                matricies, ccd_label,\
                filename_bkg,intensity_threshold,\
                boxsize,bkg_treatment,\
                filenameDirec, experimental_prefix,\
                blacklist_file, text_file, \
                files_treated,try_previous1,\
                wb, temp_key, cor_file_directory, mode_spotCycle1,\
                softmax_threshold_global123,mr_threshold_global123,\
                cap_matchrate123, tolerance_strain123, tolerance_strain1231,\
                NumberMaxofFits123,fit_peaks_gaussian_global123,\
                FitPixelDev_global123,coeff123,coeff_overlap,\
                material0_limit, material1_limit, use_previous_UBmatrix_name1,\
                    material_phase_always_present1, crystal, crystal1 = num1[ijk]

                if os.path.isfile(files):
                    try:
                        strain_matrix12, strain_matrixs12, \
                            rotation_matrix12, col12, \
                                colx12, coly12,\
                        match_rate12, mat_global12, cnt12,\
                            files_treated12, spots_len12, \
                                iR_pix12, fR_pix12, check12, best_match12 = predict_preprocessMultiProcess(files, cnt, 
                                                                   rotation_matrix,strain_matrix,strain_matrixs,
                                                                   col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,
                                                                   mat_global,
                                                                   check,detectorparameters,pixelsize,angbins,
                                                                   classhkl, hkl_all_class0, hkl_all_class1, emin, emax,
                                                                   material_, material1_, symmetry, symmetry1,lim_x,lim_y,
                                                                   strain_calculation, ind_mat, ind_mat1,
                                                                   model_direc, tolerance, tolerance1,
                                                                   matricies, ccd_label,
                                                                   filename_bkg,intensity_threshold,
                                                                   boxsize,bkg_treatment,
                                                                   filenameDirec, experimental_prefix,
                                                                   blacklist_file, text_file, 
                                                                   files_treated,try_previous1,
                                                                   wb, temp_key, cor_file_directory, mode_spotCycle1,
                                                                   softmax_threshold_global123,mr_threshold_global123,
                                                                   cap_matchrate123, tolerance_strain123,
                                                                   tolerance_strain1231,NumberMaxofFits123,
                                                                   fit_peaks_gaussian_global123,
                                                                   FitPixelDev_global123, coeff123,coeff_overlap,
                                                                   material0_limit,material1_limit,
                                                                   use_previous_UBmatrix_name1,
                                                                   material_phase_always_present1,
                                                                   crystal, crystal1)
                        files_worked.append(ijk)
                        meta['proc_id'] = proc_id
                        r_message = (strain_matrix12, strain_matrixs12, rotation_matrix12, col12, \
                                     colx12, coly12, match_rate12, mat_global12, cnt12, meta, \
                                     files_treated12, spots_len12, iR_pix12, fR_pix12, best_match12, check12)
                        outputs_queue.put(r_message)
                    except Exception as e:
                        # print(e)
                        continue
    print("broke the worker while loop")
    
def update_data_mp():
    print("ok working")
    if not _outputs_queue.empty():
        global total_progress
        
        #timermp1212.blockSignals(True)
        n_range = _outputs_queue.qsize()
        for _ in range(n_range):
            
            total_progress += 1
            progress.value = total_progress/max_progress 
            
            r_message_mpdata = _outputs_queue.get()
            strain_matrix_mpdata, strain_matrixs_mpdata, rotation_matrix_mpdata, col_mpdata, \
                             colx_mpdata, coly_mpdata, match_rate_mpdata, mat_global_mpdata, \
                                 cnt_mpdata, meta_mpdata, files_treated_mpdata, spots_len_mpdata, \
                                     iR_pixel_mpdata, fR_pixel_mpdata, best_match_mpdata, check_mpdata = r_message_mpdata

            for i_mpdata in files_treated_mpdata:
                files_treated.append(i_mpdata)
                            
            for intmat_mpdata in range(int(ubmat)):
                check[cnt_mpdata,intmat_mpdata] = check_mpdata[cnt_mpdata,intmat_mpdata]
                mat_global[intmat_mpdata][0][cnt_mpdata] = mat_global_mpdata[intmat_mpdata][0][cnt_mpdata]
                strain_matrix[intmat_mpdata][0][cnt_mpdata,:,:] = strain_matrix_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                strain_matrixs[intmat_mpdata][0][cnt_mpdata,:,:] = strain_matrixs_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                rotation_matrix[intmat_mpdata][0][cnt_mpdata,:,:] = rotation_matrix_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                col[intmat_mpdata][0][cnt_mpdata,:] = col_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                colx[intmat_mpdata][0][cnt_mpdata,:] = colx_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                coly[intmat_mpdata][0][cnt_mpdata,:] = coly_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                match_rate[intmat_mpdata][0][cnt_mpdata] = match_rate_mpdata[intmat_mpdata][0][cnt_mpdata]
                spots_len[intmat_mpdata][0][cnt_mpdata] = spots_len_mpdata[intmat_mpdata][0][cnt_mpdata]
                iR_pix[intmat_mpdata][0][cnt_mpdata] = iR_pixel_mpdata[intmat_mpdata][0][cnt_mpdata]
                fR_pix[intmat_mpdata][0][cnt_mpdata] = fR_pixel_mpdata[intmat_mpdata][0][cnt_mpdata]
                best_match[intmat_mpdata][0][cnt_mpdata] = best_match_mpdata[intmat_mpdata][0][cnt_mpdata] 
                
            if use_previous_UBmatrix_name:
                try:
                    avg_match_rate1 = [[] for i in range(int(ubmat))]
                    for intmat_mpdata in range(int(ubmat)):
                        avg_match_rate = []
                        for j in match_rate[intmat_mpdata][0][:]:
                            if j != 0:
                                avg_match_rate.append(j)
                        avg_match_rate1[intmat_mpdata].append(np.median(avg_match_rate))
                    np.savez_compressed(model_direc+'//rotation_matrix_indexed_1.npz', 
                                        rotation_matrix, mat_global, 
                                        match_rate, avg_match_rate1)
                except:
                    print("Warning : Error saving the NPZ file; nothing to worry")
        #timermp1212.blockSignals(False)
        
    
def read_hdf5(path):
    weights = {}
    keys = []
    with h5py.File(path, 'r') as f: # open file
        f.visit(keys.append) # append all keys to list
        for key in keys:
            if ':' in key: # contains data if ':' in key
                weights[f[key].name] = f[key][:]
    return weights

def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x).T, axis=0)).T

def predict(x, wb, temp_key):
    # first layer
    layer0 = np.dot(x, wb[temp_key[1]]) + wb[temp_key[0]] 
    layer0 = np.maximum(0, layer0) ## ReLU activation
    # Second layer
    layer1 = np.dot(layer0, wb[temp_key[3]]) + wb[temp_key[2]] 
    layer1 = np.maximum(0, layer1)
    # Third layer
    layer2 = np.dot(layer1, wb[temp_key[5]]) + wb[temp_key[4]]
    layer2 = np.maximum(0, layer2)
    # Output layer
    layer3 = np.dot(layer2, wb[temp_key[7]]) + wb[temp_key[6]]
    layer3 = softmax(layer3) ## output softmax activation
    return layer3

def predict_preprocessMultiProcess(files, cnt, 
                         rotation_matrix,strain_matrix,strain_matrixs,
                        col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,mat_global,
                        check,detectorparameters,pixelsize,angbins,
                        classhkl, hkl_all_class0, hkl_all_class1, emin, emax,
                        material_, material1_, symmetry, symmetry1,lim_x,lim_y,
                        strain_calculation, ind_mat, ind_mat1,
                        model_direc=None, tolerance =None, tolerance1 =None,
                       matricies=None, ccd_label=None,
                       filename_bkg=None,intensity_threshold=None,
                       boxsize=None,bkg_treatment=None,
                       filenameDirec=None, experimental_prefix=None,
                       blacklist_file =None, text_file=None, 
                       files_treated=None,try_previous1=False,
                       wb=None, temp_key=None, cor_file_directory=None, mode_spotCycle1=None,
                       softmax_threshold_global123=None,mr_threshold_global123=None,
                       cap_matchrate123=None,tolerance_strain123=None,tolerance_strain1231=None,\
                       NumberMaxofFits123=None,fit_peaks_gaussian_global123=None,
                       FitPixelDev_global123=None,coeff123=None, coeff_overlap=None,
                       material0_limit=None, material1_limit=None, use_previous_UBmatrix_name=None,
                       material_phase_always_present=None, crystal=None, crystal1=None):
    
    if files in files_treated:
        return strain_matrix, strain_matrixs, rotation_matrix, col, colx, coly, \
            match_rate, mat_global, cnt, files_treated,spots_len,iR_pix,fR_pix, check, best_match
    
    if files.split(".")[1] != "cor":
        CCDLabel=ccd_label
        seednumber = "Experimental "+CCDLabel+" file"    
        
        try:
            out_name = blacklist_file
        except:
            out_name = None  
            
        if bkg_treatment == None:
            bkg_treatment = "A-B"
            
        try:
            ### Max space = space betzeen pixles
            peak_XY = RMCCD.PeakSearch(
                                        files,
                                        stackimageindex = -1,
                                        CCDLabel=CCDLabel,
                                        NumberMaxofFits=NumberMaxofFits123,
                                        PixelNearRadius=10,
                                        removeedge=2,
                                        IntensityThreshold=intensity_threshold,
                                        local_maxima_search_method=0,
                                        boxsize=boxsize,
                                        position_definition=1,
                                        verbose=0,
                                        fit_peaks_gaussian=fit_peaks_gaussian_global123,
                                        xtol=0.001,                
                                        FitPixelDev=FitPixelDev_global123,
                                        return_histo=0,
                                        # Saturation_value=1e10,  # to be merged in CCDLabel
                                        # Saturation_value_flatpeak=1e10,
                                        MinIntensity=0,
                                        PeakSizeRange=(0.65,200),
                                        write_execution_time=1,
                                        Data_for_localMaxima = "auto_background",
                                        formulaexpression=bkg_treatment,
                                        Remove_BlackListedPeaks_fromfile=out_name,
                                        reject_negative_baseline=True,
                                        Fit_with_Data_for_localMaxima=False,
                                        maxPixelDistanceRejection=15.0,
                                        )
            peak_XY = peak_XY[0]#[:,:2] ##[2] Integer peak lists
        except:
            print("Error in Peak detection for "+ files)
            for intmat in range(matricies):
                rotation_matrix[intmat][0][cnt,:,:] = np.zeros((3,3))
                strain_matrix[intmat][0][cnt,:,:] = np.zeros((3,3))
                strain_matrixs[intmat][0][cnt,:,:] = np.zeros((3,3))
                col[intmat][0][cnt,:] = 0,0,0
                colx[intmat][0][cnt,:] = 0,0,0
                coly[intmat][0][cnt,:] = 0,0,0
                match_rate[intmat][0][cnt] = 0
                mat_global[intmat][0][cnt] = 0
                spots_len[intmat][0][cnt] = 0
                iR_pix[intmat][0][cnt] = 0
                fR_pix[intmat][0][cnt] = 0
                check[cnt,intmat] = 0
            # files_treated.append(files)
            return strain_matrix, strain_matrixs, rotation_matrix, col, colx, coly, \
                match_rate, mat_global, cnt, files_treated,spots_len,iR_pix,fR_pix, check, best_match
        
        try:
            s_ix = np.argsort(peak_XY[:, 2])[::-1]
            peak_XY = peak_XY[s_ix]
        except:
            print("Error in Peak detection for "+ files)
            for intmat in range(matricies):
                rotation_matrix[intmat][0][cnt,:,:] = np.zeros((3,3))
                strain_matrix[intmat][0][cnt,:,:] = np.zeros((3,3))
                strain_matrixs[intmat][0][cnt,:,:] = np.zeros((3,3))
                col[intmat][0][cnt,:] = 0,0,0
                colx[intmat][0][cnt,:] = 0,0,0
                coly[intmat][0][cnt,:] = 0,0,0
                match_rate[intmat][0][cnt] = 0
                mat_global[intmat][0][cnt] = 0
                spots_len[intmat][0][cnt] = 0
                iR_pix[intmat][0][cnt] = 0
                fR_pix[intmat][0][cnt] = 0
                check[cnt,intmat] = 0
            # files_treated.append(files)
            return strain_matrix, strain_matrixs, rotation_matrix, col, colx, coly, \
                match_rate, mat_global, cnt, files_treated,spots_len,iR_pix,fR_pix, check, best_match
        
        
        framedim = dictLT.dict_CCD[CCDLabel][0]
        twicetheta, chi = Lgeo.calc_uflab(peak_XY[:,0], peak_XY[:,1], detectorparameters,
                                            returnAngles=1,
                                            pixelsize=pixelsize,
                                            kf_direction='Z>0')
        data_theta, data_chi = twicetheta/2., chi
        
        framedim = dictLT.dict_CCD[CCDLabel][0]
        dict_dp={}
        dict_dp['kf_direction']='Z>0'
        dict_dp['detectorparameters']=detectorparameters
        dict_dp['detectordistance']=detectorparameters[0]
        dict_dp['detectordiameter']=pixelsize*framedim[0]
        dict_dp['pixelsize']=pixelsize
        dict_dp['dim']=framedim
        dict_dp['peakX']=peak_XY[:,0]
        dict_dp['peakY']=peak_XY[:,1]
        dict_dp['intensity']=peak_XY[:,2]
        
        CCDcalib = {"CCDLabel":CCDLabel,
                    "dd":detectorparameters[0], 
                    "xcen":detectorparameters[1], 
                    "ycen":detectorparameters[2], 
                    "xbet":detectorparameters[3], 
                    "xgam":detectorparameters[4],
                    "pixelsize": pixelsize}
        
        path = os.path.normpath(files)
        IOLT.writefile_cor(cor_file_directory+"//"+path.split(os.sep)[-1].split(".")[0], twicetheta, 
                           chi, peak_XY[:,0], peak_XY[:,1], peak_XY[:,2],
                           param=CCDcalib, sortedexit=0)
        
    elif files.split(".")[1] == "cor":
        seednumber = "Experimental COR file"
        allres = IOLT.readfile_cor(files, True)
        data_theta, data_chi, peakx, peaky, intensity = allres[1:6]
        CCDcalib = allres[-1]
        detectorparameters = allres[-2]
        # print('detectorparameters from file are: '+ str(detectorparameters))
        pixelsize = CCDcalib['pixelsize']
        CCDLabel = CCDcalib['CCDLabel']
        framedim = dictLT.dict_CCD[CCDLabel][0]
        dict_dp={}
        dict_dp['kf_direction']='Z>0'
        dict_dp['detectorparameters']=detectorparameters
        dict_dp['detectordistance']=detectorparameters[0]
        dict_dp['detectordiameter']=pixelsize*framedim[0]
        dict_dp['pixelsize']=pixelsize
        dict_dp['dim']=framedim
        dict_dp['peakX']=peakx
        dict_dp['peakY']=peaky
        dict_dp['intensity']=intensity

    sorted_data = np.transpose(np.array([data_theta, data_chi]))
    tabledistancerandom = np.transpose(GT.calculdist_from_thetachi(sorted_data, sorted_data))

    codebars_all = []
    
    if len(data_theta) == 0:
        print("No peaks Found for : " + files)
        for intmat in range(matricies):
            rotation_matrix[intmat][0][cnt,:,:] = np.zeros((3,3))
            strain_matrix[intmat][0][cnt,:,:] = np.zeros((3,3))
            strain_matrixs[intmat][0][cnt,:,:] = np.zeros((3,3))
            col[intmat][0][cnt,:] = 0,0,0
            colx[intmat][0][cnt,:] = 0,0,0
            coly[intmat][0][cnt,:] = 0,0,0
            match_rate[intmat][0][cnt] = 0
            mat_global[intmat][0][cnt] = 0
            spots_len[intmat][0][cnt] = 0
            iR_pix[intmat][0][cnt] = 0
            fR_pix[intmat][0][cnt] = 0
            check[cnt,intmat] = 0
        # files_treated.append(files)
        return strain_matrix, strain_matrixs, rotation_matrix, col, colx, coly, \
                match_rate, mat_global, cnt, files_treated,spots_len,iR_pix,fR_pix, check, best_match
    
    if not use_om_user:
        spots_in_center = np.arange(0,len(data_theta))
        spots_in_center = spots_in_center[:nb_spots_consider]
        
        for i in spots_in_center:
            spotangles = tabledistancerandom[i]
            spotangles = np.delete(spotangles, i)# removing the self distance
            # codebars = np.histogram(spotangles, bins=angbins)[0]
            codebars = histogram1d(spotangles, range=[min(angbins),max(angbins)], bins=len(angbins)-1)
            ## normalize the same way as training data
            max_codebars = np.max(codebars)
            codebars = codebars/ max_codebars
            codebars_all.append(codebars)
            
        ## reshape for the model to predict all spots at once
        codebars = np.array(codebars_all)
        ## Do prediction of all spots at once
        prediction = predict(codebars, wb, temp_key)
        
        # prediction = model.predict(codebars)
        max_pred = np.max(prediction, axis = 1)
        class_predicted = np.argmax(prediction, axis = 1)
        
        predicted_hkl123 = classhkl[class_predicted]
        predicted_hkl123 = predicted_hkl123.astype(int)
    else:
        max_pred = None
        class_predicted = None
        predicted_hkl123 = None
        spots_in_center = None
        
    s_tth = data_theta * 2.
    s_chi = data_chi
    
    rotation_matrix1, mr_highest, mat_highest, \
        strain_crystal, strain_sample, iR_pix1, \
                    fR_pix1, spots_len1,\
                    best_match1, check12 = predict_ubmatrix(seednumber, spots_in_center, classhkl, 
                                                hkl_all_class0, 
                                                hkl_all_class1, files,
                                                  s_tth1=s_tth,s_chi1=s_chi,
                                                  predicted_hkl1=predicted_hkl123,
                                                  class_predicted1=class_predicted,
                                                  max_pred1=max_pred,
                                                  emin=emin,emax=emax,
                                                  material_=material_, 
                                                  material1_=material1_, 
                                                  lim_y=lim_y, lim_x=lim_x, 
                                                  cnt=cnt,
                                                  dict_dp=dict_dp,
                                                  rotation_matrix=rotation_matrix,
                                                  mat_global=mat_global,
                                                  strain_calculation=strain_calculation,
                                                  ind_mat=ind_mat, 
                                                  ind_mat1=ind_mat1,
                                                  tolerance=tolerance, 
                                                  tolerance1 =tolerance1,
                                                  matricies=matricies,
                                                  tabledistancerandom=tabledistancerandom,
                                                  text_file = text_file,
                                                  try_previous1=try_previous1,
                                                  mode_spotCycle=mode_spotCycle1,
                                                  softmax_threshold_global123 = softmax_threshold_global123,
                                                  mr_threshold_global123=mr_threshold_global123,
                                                  cap_matchrate123=cap_matchrate123,
                                                  tolerance_strain123=tolerance_strain123,
                                                  tolerance_strain1231=tolerance_strain1231,
                                                  coeff123=coeff123,
                                                  coeff_overlap=coeff_overlap,
                                                  material0_limit=material0_limit, 
                                                  material1_limit=material1_limit,
                                                  model_direc=model_direc,
                                                  use_previous_UBmatrix_name=use_previous_UBmatrix_name,
                                                  material_phase_always_present=material_phase_always_present,
                                                  match_rate=match_rate,
                                                  check=check[cnt,:],
                                                  crystal=crystal,
                                                  crystal1=crystal1)
    
    for intmat in range(matricies):
        if len(rotation_matrix1[intmat]) == 0:
            col[intmat][0][cnt,:] = 0,0,0
            colx[intmat][0][cnt,:] = 0,0,0
            coly[intmat][0][cnt,:] = 0,0,0
        else:
            mat_global[intmat][0][cnt] = mat_highest[intmat][0]
            
            final_symm =symmetry
            final_crystal = crystal
            if mat_highest[intmat][0] == 1:
                final_symm = symmetry
                final_crystal = crystal
            elif mat_highest[intmat][0] == 2:
                final_symm = symmetry1
                final_crystal = crystal1
            symm_operator = final_crystal._hklsym
            strain_matrix[intmat][0][cnt,:,:] = strain_crystal[intmat][0]
            strain_matrixs[intmat][0][cnt,:,:] = strain_sample[intmat][0]
            rotation_matrix[intmat][0][cnt,:,:] = rotation_matrix1[intmat][0]
            col_temp = get_ipf_colour(rotation_matrix1[intmat][0], np.array([0., 0., 1.]), final_symm, symm_operator)
            col[intmat][0][cnt,:] = col_temp
            col_tempx = get_ipf_colour(rotation_matrix1[intmat][0], np.array([1., 0., 0.]), final_symm, symm_operator)
            colx[intmat][0][cnt,:] = col_tempx
            col_tempy = get_ipf_colour(rotation_matrix1[intmat][0], np.array([0., 1., 0.]), final_symm, symm_operator)
            coly[intmat][0][cnt,:] = col_tempy
            match_rate[intmat][0][cnt] = mr_highest[intmat][0]
            spots_len[intmat][0][cnt] = spots_len1[intmat][0]
            iR_pix[intmat][0][cnt] = iR_pix1[intmat][0]
            fR_pix[intmat][0][cnt] = fR_pix1[intmat][0]
            best_match[intmat][0][cnt] = best_match1[intmat][0]
            check[cnt,intmat] = check12[intmat]

    files_treated.append(files)
    return strain_matrix, strain_matrixs, rotation_matrix, col, colx, coly, match_rate, \
            mat_global, cnt, files_treated, spots_len, iR_pix, fR_pix, check, best_match

def get_material_dataP(Gstar, classhkl = None):
    hkl2 = np.copy(classhkl)
    hkl1 = np.copy(classhkl)
    # compute square matrix containing angles
    metrics = Gstar
    H1 = hkl1
    n1 = hkl1.shape[0]
    H2 = hkl2
    n2 = hkl2.shape[0]
    dstar_square_1 = np.diag(np.inner(np.inner(H1, metrics), H1))
    dstar_square_2 = np.diag(np.inner(np.inner(H2, metrics), H2))
    scalar_product = np.inner(np.inner(H1, metrics), H2) * 1.0
    d1 = np.sqrt(dstar_square_1.reshape((n1, 1))) * 1.0
    d2 = np.sqrt(dstar_square_2.reshape((n2, 1))) * 1.0
    outy = np.outer(d1, d2)
    ratio = scalar_product / outy
    ratio = np.round(ratio, decimals=7)
    tab_angulardist = np.arccos(ratio) / (np.pi / 180.0)
    np.putmask(tab_angulardist, np.abs(tab_angulardist) < 0.001, 400)
    return tab_angulardist

def predict_ubmatrix(seednumber, spots_in_center, classhkl, hkl_all_class0, 
                     hkl_all_class1, filename, 
                     s_tth1,s_chi1,predicted_hkl1,class_predicted1,max_pred1,
                     emin, emax, material_, material1_, lim_y, lim_x, cnt,
                     dict_dp,rotation_matrix,mat_global,strain_calculation,
                     ind_mat, ind_mat1,
                     tolerance=None,  tolerance1 =None, matricies=None, tabledistancerandom=None,
                     text_file=None,try_previous1=False, mode_spotCycle=None,
                     softmax_threshold_global123=None,mr_threshold_global123=None,
                     cap_matchrate123=None, tolerance_strain123=None,tolerance_strain1231=None, coeff123=None,
                     coeff_overlap=None, material0_limit=None, material1_limit=None, model_direc=None,
                     use_previous_UBmatrix_name=None, material_phase_always_present=None, match_rate=None,
                     check = None, crystal=None, crystal1=None):
    
    print("# Predicting for "+ filename)
    input_params = {"tolerance": tolerance,
                     "tolerance1":tolerance1,
                    "tolerancestrain": tolerance_strain123, ## For strain calculations
                    "tolerancestrain1": tolerance_strain1231,
                    "emin": emin,
                    "emax": emax,
                    "mat":0}
    
    strain_matrix = [[] for i in range(matricies)]
    strain_matrixs = [[] for i in range(matricies)]
    best_matrix = [[] for i in range(matricies)]
    mr_highest = [[] for i in range(matricies)]
    ir_pixels = [[] for i in range(matricies)]
    fr_pixels = [[] for i in range(matricies)]
    spots_len = [[] for i in range(matricies)]
    mat_highest = [[] for i in range(matricies)]
    best_match = [[] for i in range(matricies)]
    spots1 = []
    spots1_global = [[] for i in range(matricies)]
    
    if not use_om_user: 
        dist = tabledistancerandom        
        ## one time calculations
        lattice_params0 = dictLT.dict_Materials[material_][1]
        B0 = CP.calc_B_RR(lattice_params0)
        Gstar_metric0 = CP.Gstar_from_directlatticeparams(lattice_params0[0],lattice_params0[1],\
                                                         lattice_params0[2],lattice_params0[3],\
                                                             lattice_params0[4],lattice_params0[5])
        tab_distance_classhkl_data0 = get_material_dataP(Gstar_metric0, predicted_hkl1[:nb_spots_consider,:])
        
        if material_ != material1_:
            lattice_params1 = dictLT.dict_Materials[material1_][1]
            B1 = CP.calc_B_RR(lattice_params1)
            Gstar_metric1 = CP.Gstar_from_directlatticeparams(lattice_params1[0],lattice_params1[1],\
                                                             lattice_params1[2],lattice_params1[3],\
                                                                 lattice_params1[4],lattice_params1[5])
            tab_distance_classhkl_data1 = get_material_dataP(Gstar_metric1, predicted_hkl1)
        else:
            tab_distance_classhkl_data1 = None
            Gstar_metric1 = None
            B1 = None
    else:
        dist = tabledistancerandom 
        tab_distance_classhkl_data0 = None
        tab_distance_classhkl_data1 = None
        ## one time calculations
        lattice_params0 = dictLT.dict_Materials[material_][1]
        B0 = CP.calc_B_RR(lattice_params0)
        Gstar_metric0 = CP.Gstar_from_directlatticeparams(lattice_params0[0],lattice_params0[1],\
                                                         lattice_params0[2],lattice_params0[3],\
                                                             lattice_params0[4],lattice_params0[5])        
        if material_ != material1_:
            lattice_params1 = dictLT.dict_Materials[material1_][1]
            B1 = CP.calc_B_RR(lattice_params1)
            Gstar_metric1 = CP.Gstar_from_directlatticeparams(lattice_params1[0],lattice_params1[1],\
                                                             lattice_params1[2],lattice_params1[3],\
                                                                 lattice_params1[4],lattice_params1[5])
        else:
            Gstar_metric1 = None
            B1 = None
        
    spots = []
    first_match = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, np.zeros((3,3))]
    max_mr = 0
    mat = 0
    iR = 0
    fR = 0
    strain_crystal = np.zeros((3,3))
    strain_sample = np.zeros((3,3))
    material0_count = 0
    material1_count = 0
    calcul_done = False
    objective_function1 = None
    
    for igrain in range(matricies):
        # if check[igrain] == 1: # or len(spots1_global[igrain]) != 0:
        #     continue
        
        try_previous = try_previous1
        max_mr, min_mr = 0, 0
        iR, fR= 0, 0
        case = "None"
        
        if use_om_user:
            use_previous_UBmatrix_name = False
            try_previous = False

            temp_qsd = np.loadtxt(r"C:\Users\purushot\Desktop\Al2TiO5_laue\config_files\rot_mat_VF_P1_R4.txt", delimiter=",")
            temp_qsd = temp_qsd.reshape((len(temp_qsd),3,3))
            rotationmatrix_indexed = temp_qsd[igrain,:,:]
            
            mat = 1

            if mat == 1:
                Keymaterial_ = material_
                case = material_
                Bkey = B0
                input_params["mat"] = 1
                input_params["Bmat"] = Bkey
            elif mat == 2:
                Keymaterial_ = material1_
                case = material1_
                Bkey = B1
                input_params["mat"] = 2
                input_params["Bmat"] = Bkey
                
            spots_prev, theo_spots_prev = remove_spots(s_tth1, s_chi1, rotationmatrix_indexed, 
                                                         Keymaterial_, input_params, dict_dp['detectorparameters'],
                                                         dict_dp)
            newmatchrate = 100*len(spots_prev)/theo_spots_prev
            
            ## Filter indexation by matching rate
            if newmatchrate < cap_matchrate123:
                strain_crystal, strain_sample, iR, fR = np.zeros((3,3)), np.zeros((3,3)), 0, 0
                first_match = [0, 0, 0, 0, 0, 0, 0, 0, 0, \
                                    0, 0, 0, 0, 0, np.zeros((3,3))]
                spots = []
                max_mr, min_mr = 0, 0
            else:
                if strain_calculation:
                    strain_crystal, strain_sample, iR, fR, rot_mat_UB = calculate_strains_fromUB(s_tth1, s_chi1, 
                                                                                  rotationmatrix_indexed,
                                                                                  Keymaterial_, 
                                                                                 input_params, dict_dp['detectorparameters'], 
                                                                                 dict_dp, spots1, Bkey)
                else:
                    strain_crystal, strain_sample, iR, fR = np.zeros((3,3)), np.zeros((3,3)), 0, 0
                    rot_mat_UB = np.copy(rotationmatrix_indexed)
                spots = spots_prev
                expected = theo_spots_prev
                max_mr, min_mr = 100*(len(spots)/expected), 100*(len(spots)/expected)
                first_match = [0, 0, 0, 0, 0, 0, 0, 0, 0, \
                                0, len(spots), expected, max_mr, 0, rot_mat_UB]
                    
            try_previous = False
            calcul_done = True

        
        elif use_previous_UBmatrix_name:
            try:
                try_previous = False
                ### try already indexed UB matricies
                # xy = np.load('xy.npz')
                # xy.zip.fp.close()
                # xy.close()
                with np.load(model_direc+"//rotation_matrix_indexed_1.npz") as load_objectind:
                    # load_objectind = np.load(model_direc+"//rotation_matrix_indexed.npz")
                    rotationmatrix_indexed = load_objectind["arr_0"]
                    mat_global_indexed = load_objectind["arr_1"]
                    match_rate_indexed = load_objectind["arr_2"]
                    avg_match_rate_indexed = load_objectind["arr_3"]
                calcul_done = False
                for ind_mat_UBmat in range(len(rotationmatrix_indexed[igrain][0])):
                    if calcul_done:
                        continue
                    
                    if np.all(rotationmatrix_indexed[igrain][0][ind_mat_UBmat,:,:]) == 0:
                        continue

                    if match_rate_indexed[igrain][0][ind_mat_UBmat] < 0.8*avg_match_rate_indexed[igrain]:
                        continue
                    
                    mat = mat_global_indexed[igrain][0][ind_mat_UBmat]
                    if mat == 1:
                        Keymaterial_ = material_
                        case = material_
                        Bkey = B0
                        input_params["mat"] = 1
                        input_params["Bmat"] = Bkey
                    elif mat == 2:
                        Keymaterial_ = material1_
                        case = material1_
                        Bkey = B1
                        input_params["mat"] = 2
                        input_params["Bmat"] = Bkey
                    else:
                        Keymaterial_ = None
                        Bkey = None
                        input_params["mat"] = 0
                        input_params["Bmat"] = None
                        continue
                    
                    spots_prev, theo_spots_prev = remove_spots(s_tth1, s_chi1, rotationmatrix_indexed[igrain][0][ind_mat_UBmat,:,:], 
                                                                 Keymaterial_, input_params, dict_dp['detectorparameters'],
                                                                 dict_dp)
                    newmatchrate = 100*len(spots_prev)/theo_spots_prev
                    condition_prev = newmatchrate < 0.8*(match_rate_indexed[igrain][0][ind_mat_UBmat])
                    current_spots = [len(list(set(spots_prev) & set(spots1_global[igr]))) > coeff_overlap*len(spots1_global[igr]) for igr in range(len(spots1_global))]
                    
                    if condition_prev or (newmatchrate <= cap_matchrate123) or np.any(current_spots):# or overlap:
                        try_previous = try_previous1
                    else:
                        try_previous = False
                        calcul_done = True
                        if strain_calculation:
                            strain_crystal, strain_sample, iR, fR, rot_mat_UB = calculate_strains_fromUB(s_tth1, s_chi1, 
                                                                                          rotationmatrix_indexed[igrain][0][ind_mat_UBmat,:,:],
                                                                                          Keymaterial_, 
                                                                                         input_params, dict_dp['detectorparameters'], 
                                                                                         dict_dp, spots1, Bkey)
                        else:
                            strain_crystal, strain_sample, iR, fR = np.zeros((3,3)), np.zeros((3,3)), 0, 0
                            rot_mat_UB = np.copy(rotationmatrix_indexed[igrain][0][ind_mat_UBmat,:,:])
                        spots = spots_prev
                        expected = theo_spots_prev
                        max_mr, min_mr = 100*(len(spots)/expected), 100*(len(spots)/expected)
                        first_match = [0, 0, 0, 0, 0, 0, 0, 0, 0, \
                                        0, len(spots), expected, max_mr, 0, rot_mat_UB]
                        break
            except:
                try_previous = False
                calcul_done = False
        
        if not try_previous and not calcul_done:
            if mode_spotCycle == "graphmode_allHKL":
                # print("Fast mode of analysis")
                first_match, max_mr, min_mr, spots, \
                        case, mat, strain_crystal, \
                            strain_sample, iR, fR,objective_function1  = get_orient_mat_graphv1(s_tth1, s_chi1,
                                                                        material_, material1_, classhkl,
                                                                        class_predicted1, predicted_hkl1,
                                                                        input_params, hkl_all_class0, hkl_all_class1,
                                                                        max_pred1, dict_dp, 
                                                                        spots1, dist, 
                                                                        Gstar_metric0, Gstar_metric1, B0, B1,
                                                                        softmax_threshold=softmax_threshold_global123,
                                                                        mr_threshold=mr_threshold_global123,
                                                                        tab_distance_classhkl_data0=tab_distance_classhkl_data0,
                                                                        tab_distance_classhkl_data1=tab_distance_classhkl_data1,
                                                                        spots1_global = spots1_global,
                                                                        coeff_overlap = coeff_overlap,
                                                                        ind_mat=ind_mat, ind_mat1=ind_mat1, 
                                                                        strain_calculation=strain_calculation,
                                                                        cap_matchrate123=cap_matchrate123,
                                                                        material0_count=material0_count,
                                                                        material1_count=material1_count,
                                                                        material0_limit=material0_limit,
                                                                        material1_limit=material1_limit,
                                                                        igrain=igrain,
                                                                        material_phase_always_present=material_phase_always_present,
                                                                        objective_function= objective_function1,
                                                                        crystal=crystal,
                                                                        crystal1=crystal1)                
        for ispot in spots:
            spots1.append(ispot)
            spots1_global[igrain].append(ispot)

        ## make copy of best rotation matrix
        best_match[igrain].append(np.copy(first_match))
        best_matrix[igrain].append(np.copy(first_match[14]))
        mr_highest[igrain].append(np.copy(max_mr))
        mat_highest[igrain].append(np.copy(mat))
        ir_pixels[igrain].append(np.copy(iR))
        fr_pixels[igrain].append(np.copy(fR))
        spots_len[igrain].append(np.copy(len(spots)))
        strain_matrix[igrain].append(np.copy(strain_crystal))
        strain_matrixs[igrain].append(np.copy(strain_sample))
        
        if np.all(first_match[14] != 0):
            check[igrain] = 1
        
        if mat == 1:
            material0_count += 1
        if mat == 2:
            material1_count += 1
    return best_matrix, mr_highest, mat_highest, strain_matrix, strain_matrixs, ir_pixels, fr_pixels, spots_len, best_match, check


def get_orient_mat_graphv1(s_tth, s_chi, material0_, material1_, classhkl, class_predicted, predicted_hkl,
                       input_params, hkl_all_class0, hkl_all_class1, max_pred, dict_dp, spots, 
                       dist, Gstar_metric0, Gstar_metric1, B0, B1, softmax_threshold=0.85, mr_threshold=0.85, 
                       tab_distance_classhkl_data0=None, tab_distance_classhkl_data1=None, spots1_global=None,
                       coeff_overlap = None, ind_mat=None, ind_mat1=None, strain_calculation=None, cap_matchrate123=None,
                       material0_count=None, material1_count=None, material0_limit=None, material1_limit=None,
                       igrain=None, material_phase_always_present=None, objective_function=None, crystal=None,
                       crystal1=None):
    
    if objective_function == None:
        init_mr = 0
        init_mat = 0
        init_material = "None"
        init_case = "None"
        init_B = None
        final_match_rate = 0
        match_rate_mma = []
        final_rmv_ind = []

        if material0_ == material1_:
            list_of_sets = []
            for ii in range(nb_spots_consider):#0, min(nb_spots_consider, len(dist))):
                if max_pred[ii] < softmax_threshold:
                    continue 
                a1 = np.round(dist[ii],3)
                
                for i in range(nb_spots_consider):#len(dist)):
                    if ii==i:
                        continue
                    if (ii,i) in list_of_sets or (i,ii) in list_of_sets:
                        continue
                    
                    if max_pred[i] < softmax_threshold:
                        continue
                    hkl1 = hkl_all_class0[str(predicted_hkl[ii])]
                    hkl1_list = np.array(hkl1)
                    hkl2 = hkl_all_class0[str(predicted_hkl[i])]
                    hkl2_list = np.array(hkl2)
                    Gstar_metric = Gstar_metric0
                    
                    tab_angulardist_temp = CP.AngleBetweenNormals(hkl1_list, hkl2_list, Gstar_metric)
                    np.putmask(tab_angulardist_temp, np.abs(tab_angulardist_temp) < 0.001, 400)
                    
                    list_ = np.where(np.abs(tab_angulardist_temp-a1[i]) < input_params["tolerance"])
                    if len(list_[0]) != 0:
                        list_of_sets.append((ii,i))

        else:
            list_of_sets = []
            for ii in range(0, min(nb_spots_consider, len(dist))):
                if max_pred[ii] < softmax_threshold:
                    continue 
                
                a1 = np.round(dist[ii],3)

                for i in range(len(dist)):
                    if ii==i:
                        continue
                    if (ii,i) in list_of_sets or (i,ii) in list_of_sets:
                        continue
                    
                    if max_pred[i] < softmax_threshold:
                        continue
                    if class_predicted[ii] < ind_mat and class_predicted[i] < ind_mat:
                        tab_distance_classhkl_data = tab_distance_classhkl_data0
                        tolerance_new = input_params["tolerance"]
                        hkl1 = hkl_all_class0[str(predicted_hkl[ii])]
                        hkl1_list = np.array(hkl1)
                        hkl2 = hkl_all_class0[str(predicted_hkl[i])]
                        hkl2_list = np.array(hkl2)
                        Gstar_metric = Gstar_metric0
                        
                    elif (ind_mat <= class_predicted[ii] < (ind_mat+ind_mat1)) and \
                                        (ind_mat <= class_predicted[i] < (ind_mat+ind_mat1)):
                        tab_distance_classhkl_data = tab_distance_classhkl_data1
                        tolerance_new = input_params["tolerance1"]
                        hkl1 = hkl_all_class1[str(predicted_hkl[ii])]
                        hkl1_list = np.array(hkl1)
                        hkl2 = hkl_all_class1[str(predicted_hkl[i])]
                        hkl2_list = np.array(hkl2)
                        Gstar_metric = Gstar_metric1
                        
                    else:
                        continue
                    
                    tab_angulardist_temp = CP.AngleBetweenNormals(hkl1_list, hkl2_list, Gstar_metric)
                    np.putmask(tab_angulardist_temp, np.abs(tab_angulardist_temp) < 0.001, 400)
                    list_ = np.where(np.abs(tab_angulardist_temp-a1[i]) < tolerance_new)
                    if len(list_[0]) != 0:
                        list_of_sets.append((ii,i))

        ## build a direct connection graph object
        graph_obj = nx.DiGraph(list_of_sets)
        connected_nodes_length = []
        connected_nodes = [[] for i in range(len(graph_obj))]
        for i,line in enumerate(nx.generate_adjlist(graph_obj)):
            connected_nodes_length.append(len(line.split(" ")))
            connected_nodes[i].append([int(jj) for jj in line.split(" ")])
        
        ## sort by maximum node occurance
        connected_nodes_length = np.array(connected_nodes_length)
        connected_nodes_length_sort_ind = np.argsort(connected_nodes_length)[::-1]
  
        mat = 0
        case = "None"
        tried_spots = []
        
        objective_function = []
        for toplist in range(len(graph_obj)):
            # ## continue if less than 3 connections are found for a graph
            # if connected_nodes_length[connected_nodes_length_sort_ind[toplist]] < 2:
            #     continue
            
            for j in connected_nodes[connected_nodes_length_sort_ind[toplist]][0]:
                init_mr = 0
                final_match_rate = 0
                final_rmv_ind = []
                all_stats = []
                for i in connected_nodes[connected_nodes_length_sort_ind[toplist]][0]:
                    if j == i:
                        continue
                    
                    if j in tried_spots and i in tried_spots:
                        continue
                    
                    if material0_ == material1_:
                        tab_distance_classhkl_data = tab_distance_classhkl_data0
                        hkl_all_class = hkl_all_class0
                        material_ = material0_
                        B = B0
                        Gstar_metric = Gstar_metric0
                        case = material_
                        mat = 1
                        input_params["mat"] = mat
                        input_params["Bmat"] = B
                    else:
                        if class_predicted[i] < ind_mat and class_predicted[j] < ind_mat:
                            tab_distance_classhkl_data = tab_distance_classhkl_data0
                            hkl_all_class = hkl_all_class0
                            material_ = material0_
                            B = B0
                            Gstar_metric = Gstar_metric0
                            case = material_
                            mat = 1
                            input_params["mat"] = mat
                            input_params["Bmat"] = B
                        elif (ind_mat <= class_predicted[i] < (ind_mat+ind_mat1)) and \
                                            (ind_mat <= class_predicted[j] < (ind_mat+ind_mat1)):
                            tab_distance_classhkl_data = tab_distance_classhkl_data1
                            hkl_all_class = hkl_all_class1
                            material_ = material1_
                            B = B1
                            Gstar_metric = Gstar_metric1
                            case = material_  
                            mat = 2
                            input_params["mat"] = mat
                            input_params["Bmat"] = B
                        else:
                            mat = 0
                            case = "None"
                            input_params["mat"] = mat
                            input_params["Bmat"] = None
                    
                    if mat == 0:
                        continue                    
                    
                    tth_chi_spot1 = np.array([s_tth[i], s_chi[i]])
                    tth_chi_spot2 = np.array([s_tth[j], s_chi[j]])         
        
                    hkl1 = hkl_all_class[str(predicted_hkl[i])]
                    hkl1_list = np.array(hkl1)
                    hkl2 = hkl_all_class[str(predicted_hkl[j])]
                    hkl2_list = np.array(hkl2)
                    actual_mat, flagAM, \
                    spot1_hkl, spot2_hkl = propose_UB_matrix(hkl1_list, hkl2_list, 
                                                            Gstar_metric, input_params, 
                                                            dist[i,j],
                                                            tth_chi_spot1, tth_chi_spot2, 
                                                            B, method=0, crystal=crystal,
                                                            crystal1=crystal1)
                    
                    if flagAM:
                        continue
                    
                    for iind in range(len(actual_mat)):
                        rot_mat123 = actual_mat[iind]

                        rmv_ind, theospots = remove_spots(s_tth, s_chi, rot_mat123, 
                                                                material_, input_params, 
                                                                dict_dp['detectorparameters'], dict_dp)
                        
                        match_rate = np.round(100 * len(rmv_ind)/theospots, 3)
                        
                        match_rate_mma.append(match_rate)

                        if match_rate > init_mr:
                            final_rmv_ind = rmv_ind                    
                            init_mat = np.copy(mat)
                            input_params["mat"] = init_mat
                            init_material = np.copy(material_)
                            init_case = np.copy(case)
                            init_B = np.copy(B)  
                            input_params["Bmat"] = init_B                                     
                            final_match_rate = np.copy(match_rate)
                            init_mr = np.copy(match_rate)                   
                            all_stats = [i, j, \
                                         spot1_hkl[iind], spot2_hkl[iind], \
                                        tth_chi_spot1, tth_chi_spot2, \
                                        dist[i,j], tab_distance_classhkl_data[i,j], np.round(max_pred[i]*100,3), \
                                        np.round(max_pred[j]*100,3), len(rmv_ind), theospots,\
                                        match_rate, 0.0, rot_mat123, init_mat, init_material, init_B, init_case]
                    tried_spots.append(i)                 
                    
                if (final_match_rate <= cap_matchrate123): ## Nothing found!! 
                    ## Either peaks are not well defined or not found within tolerance and prediction accuracy
                    all_stats = [0, 0, 0, 0, 0, 0, 0, 0, 0, \
                                        0, 0, 0, 0, 0, np.zeros((3,3))]
                    max_mr, min_mr = 0, 0
                    spot_ind = []
                    mat = 0
                    input_params["mat"] = 0
                    case = "None"
                    objective_function.append([0, [], []])
                else:
                    objective_function.append([final_match_rate, final_rmv_ind, all_stats])     
                tried_spots.append(j)
 
    sort_ind = []
    for ijk in objective_function:
        sort_ind.append(ijk[0])
    sort_ind = np.array(sort_ind)
    sort_ind = np.argsort(sort_ind)[::-1]
    
    for gr_count123 in range(len(sort_ind)):           
        max_mr = objective_function[sort_ind[gr_count123]][0]
        rmv_ind = objective_function[sort_ind[gr_count123]][1]
        all_stats = objective_function[sort_ind[gr_count123]][2]
        
        if len(rmv_ind) == 0 or max_mr==0:
            continue
        
        mat = all_stats[15]
        if mat == 1:
            if igrain==0 and material_phase_always_present ==2:
                mat = 0
                case="None"
            if material0_count >= material0_limit:
                mat = 0
                case="None"
        elif mat == 2:
            if igrain==0 and material_phase_always_present ==1:
                mat = 0
                case="None"
            if material1_count >= material1_limit:
                mat = 0
                case="None"
        
        if mat == 0:
            continue

        current_spots = [len(list(set(rmv_ind) & set(spots1_global[igr])))> coeff_overlap*len(spots1_global[igr]) for igr in range(len(spots1_global))]
        
        if np.any(current_spots):
            continue
                  
        input_params["mat"] = all_stats[15]
        if strain_calculation:
            dev_strain, strain_sample, iR, fR, rot_mat_UB = calculate_strains_fromUB(s_tth, s_chi, all_stats[14], str(all_stats[16]), 
                                                                 input_params, dict_dp['detectorparameters'], 
                                                                 dict_dp, spots, all_stats[17])
        else:
            dev_strain, strain_sample, iR, fR = np.zeros((3,3)), np.zeros((3,3)), 0, 0
            rot_mat_UB = np.copy(all_stats[14])
        all_stats[14] = rot_mat_UB     
        
        return all_stats, np.max(max_mr), np.min(max_mr), \
                rmv_ind, str(all_stats[18]), all_stats[15], dev_strain, strain_sample, iR, fR, objective_function
    
    all_stats = [0, 0, 0, 0, 0, 0, 0, 0, 0, \
                        0, 0, 0, 0, 0, np.zeros((3,3))]
    max_mr, min_mr = 0, 0
    spot_ind = []
    mat = 0
    input_params["mat"] = 0
    case = "None"
    return all_stats, max_mr, min_mr, spot_ind, case, mat, np.zeros((3,3)), np.zeros((3,3)), 0, 0, objective_function

def propose_UB_matrix(hkl1_list, hkl2_list, Gstar_metric, input_params, dist123,
                      tth_chi_spot1, tth_chi_spot2, B, method=0, crystal=None,
                      crystal1=None):
    
    if method == 0:
        tab_angulardist_temp = CP.AngleBetweenNormals(hkl1_list, hkl2_list, Gstar_metric)
        
        if input_params["mat"] == 1:
            list_ = np.where(np.abs(tab_angulardist_temp-dist123) < input_params["tolerance"])
            
        elif input_params["mat"] == 2:
            list_ = np.where(np.abs(tab_angulardist_temp-dist123) < input_params["tolerance1"])

        if len(list_[0]) == 0:
            return None, True, 0, 0

        rot_mat_abs = []
        actual_mat = []
        spot1_hkl = []
        spot2_hkl = []
        
        triedspots = []
        for ii, jj in zip(list_[0], list_[1]):
            if ii in triedspots and jj in triedspots:
                continue

            conti_ = False
            
            try:
                rot_mat1 = FindO.OrientMatrix_from_2hkl(hkl1_list[ii], tth_chi_spot1, \
                                                        hkl2_list[jj], tth_chi_spot2,
                                                        B)
            except:
                continue                    
            
            copy_rm = np.copy(rot_mat1)
            copy_rm = np.round(np.abs(copy_rm),5)
            copy_rm.sort(axis=1)
            for iji in rot_mat_abs:
                iji.sort(axis=1)                        
                if np.all(iji==copy_rm):
                    conti_ = True
                    break
            if conti_:
                continue
            rot_mat_abs.append(np.round(np.abs(rot_mat1),5))
            actual_mat.append(rot_mat1)
            spot1_hkl.append(hkl1_list[ii])
            spot2_hkl.append(hkl2_list[jj])
            triedspots.append(ii)
            triedspots.append(jj)
    else:  
        # method 2
        hkl_all = np.vstack((hkl1_list, hkl2_list))
        LUT = FindO.GenerateLookUpTable(hkl_all, Gstar_metric)
        if input_params["mat"] == 1:
            hkls = FindO.PlanePairs_2(dist123, input_params["tolerance"], LUT, onlyclosest=1)
        elif input_params["mat"] == 2:
            hkls = FindO.PlanePairs_2(dist123, input_params["tolerance1"], LUT, onlyclosest=1)            
         
        if np.all(hkls == None):
            return None, True, 0, 0
                
        rot_mat_abs = []
        actual_mat = []
        spot1_hkl = []
        spot2_hkl = []
        
        for ii in range(len(hkls)):
            if np.all(hkls[ii][0] == hkls[ii][1]):
                continue
            conti_ = False
            
            try:
                rot_mat1 = FindO.OrientMatrix_from_2hkl(hkls[ii][0], tth_chi_spot1, \
                                                        hkls[ii][1], tth_chi_spot2,
                                                        B)
            except:
                continue                    
            
            copy_rm = np.copy(rot_mat1)
            copy_rm = np.round(np.abs(copy_rm),5)
            copy_rm.sort(axis=1)
            for iji in rot_mat_abs:
                iji.sort(axis=1)
                if np.all(iji==copy_rm):
                    conti_ = True
                    break

            if conti_:
                continue
            rot_mat_abs.append(np.round(np.abs(rot_mat1),5))
            actual_mat.append(rot_mat1)
            spot1_hkl.append(hkls[ii][0])
            spot2_hkl.append(hkls[ii][1])
    
    ## just fixing a* to x seems ok; if not think of aligning b* to xy plane
    sum_sign = []
    for nkl in range(len(actual_mat)):
        temp_mat = np.dot(actual_mat[nkl], B)
        ## fix could be to choose a matrix that aligns best the b* vector to Y axis or a* to X axis
        # if np.argmax(np.abs(temp_mat[:2,0])) == 0 and \
        #         np.argmax(np.abs(temp_mat[:2,1])) == 1: ##a* along x, b*along y
        if np.argmax(np.abs(temp_mat[:2,0])) == 0: ##a* along x
            sum_sign.append(2)
        elif np.argmax(np.abs(temp_mat[:2,0])) ==  np.argmax(np.abs(temp_mat[:2,1])):
            sum_sign.append(0)
        else:
            sum_sign.append(1)
    ind_sort = np.argsort(sum_sign)[::-1]
    ## re-arrange
    actual_mat1 = []
    spot1_hkl1, spot2_hkl1 = [], []
    for inin in ind_sort:
        actual_mat1.append(actual_mat[inin])
        spot1_hkl1.append(spot1_hkl[inin])
        spot2_hkl1.append(spot2_hkl[inin])
    actual_mat, spot1_hkl, spot2_hkl = actual_mat1, spot1_hkl1, spot2_hkl1
    return actual_mat, False, spot1_hkl, spot2_hkl

def remove_spots(s_tth, s_chi, first_match123, material_, input_params, detectorparameters, dict_dp):
    try:
        grain = CP.Prepare_Grain(material_, first_match123, dictmaterials=dictLT.dict_Materials)
    except:
        return [], 100
    #### Perhaps better than SimulateResult function
    kf_direction = dict_dp["kf_direction"]
    detectordistance = dict_dp["detectorparameters"][0]
    detectordiameter = dict_dp["detectordiameter"]
    pixelsize = dict_dp["pixelsize"]
    dim = dict_dp["dim"]
           
    spots2pi = LT.getLaueSpots(CST_ENERGYKEV / input_params["emax"], CST_ENERGYKEV / input_params["emin"],
                                    [grain],
                                    fastcompute=1,
                                    verbose=0,
                                    kf_direction=kf_direction,
                                    ResolutionAngstrom=False,
                                    dictmaterials=dictLT.dict_Materials)

    TwicethetaChi = LT.filterLaueSpots_full_np(spots2pi[0][0], None, onlyXYZ=False,
                                                    HarmonicsRemoval=0,
                                                    fastcompute=1,
                                                    kf_direction=kf_direction,
                                                    detectordistance=detectordistance,
                                                    detectordiameter=detectordiameter,
                                                    pixelsize=pixelsize,
                                                    dim=dim)
    ## get proximity for exp and theo spots
    if input_params["mat"] == 1:
        angtol = input_params["tolerance"]
    elif input_params["mat"] == 2:
        angtol = input_params["tolerance1"]
    else:
        return [], 100
    
    if option_global =="v1":
        # print("entering v1")
        List_Exp_spot_close, residues_link, _ = getProximityv1(np.array([TwicethetaChi[0], TwicethetaChi[1]]),  # warning array(2theta, chi)
                                                  s_tth/2.0, s_chi,  # warning theta, chi for exp
                                                  angtol=angtol)
    elif option_global =="v2":
        List_Exp_spot_close, residues_link, _ = getProximityv1_ambigious(np.array([TwicethetaChi[0], TwicethetaChi[1]]),  # warning array(2theta, chi)
                                                  s_tth/2.0, s_chi,  # warning theta, chi for exp
                                                  angtol=angtol)
    else:
        List_Exp_spot_close, residues_link, _ = getProximityv1_ambigious(np.array([TwicethetaChi[0], TwicethetaChi[1]]),  # warning array(2theta, chi)
                                                  s_tth/2.0, s_chi,  # warning theta, chi for exp
                                                  angtol=angtol)
        List_Exp_spot_close, ind_uniq = np.unique(List_Exp_spot_close, return_index=True)
        residues_link = np.take(residues_link, ind_uniq)

    if np.average(residues_link) > residues_threshold:
        return [], 100
    
    if len(np.unique(List_Exp_spot_close)) < nb_spots_global_threshold:
        return [], 100
    
    return List_Exp_spot_close, len(TwicethetaChi[0])

def getProximityv1_ambigious(TwicethetaChi, data_theta, data_chi, angtol=0.5):
    # theo simul data
    theodata = np.array([TwicethetaChi[0] / 2.0, TwicethetaChi[1]]).T
    # exp data
    sorted_data = np.array([data_theta, data_chi]).T
    table_dist = GT.calculdist_from_thetachi(sorted_data, theodata)
    prox_table = np.argmin(table_dist, axis=1)
    allresidues = np.amin(table_dist, axis=1)
    very_close_ind = np.where(allresidues < angtol)[0]
    List_Exp_spot_close = []
    theo_index = []
    if len(very_close_ind) > 0:
        for theospot_ind in very_close_ind:  # loop over theo spots index
            List_Exp_spot_close.append(prox_table[theospot_ind])
            theo_index.append(theospot_ind)
    return List_Exp_spot_close, allresidues[very_close_ind], theo_index

def getProximityv1( TwicethetaChi, data_theta, data_chi, angtol=0.5):
    theodata = np.array([TwicethetaChi[0] / 2.0, TwicethetaChi[1]]).T
    # exp data
    sorted_data = np.array([data_theta, data_chi]).T
    table_dist = GT.calculdist_from_thetachi(sorted_data, theodata)

    prox_table = np.argmin(table_dist, axis=1)
    allresidues = np.amin(table_dist, axis=1)
    very_close_ind = np.where(allresidues < angtol)[0]
    List_Exp_spot_close = []
    Miller_Exp_spot = []
    if len(very_close_ind) > 0:
        for theospot_ind in very_close_ind:  # loop over theo spots index
            List_Exp_spot_close.append(prox_table[theospot_ind])
            Miller_Exp_spot.append(1)
    else:
        return [], [], []
    # removing exp spot which appears many times(close to several simulated spots of one grain)--------------
    arrayLESC = np.array(List_Exp_spot_close, dtype=float)
    sorted_LESC = np.sort(arrayLESC)
    diff_index = sorted_LESC - np.array(list(sorted_LESC[1:]) + [sorted_LESC[0]])
    toremoveindex = np.where(diff_index == 0)[0]
    if len(toremoveindex) > 0:
        # index of exp spot in arrayLESC that are duplicated
        ambiguous_exp_ind = GT.find_closest(np.array(sorted_LESC[toremoveindex], dtype=float), arrayLESC, 0.1)[1]
        for ind in ambiguous_exp_ind:
            Miller_Exp_spot[ind] = None
    
    ProxTablecopy = np.copy(prox_table)

    for theo_ind, exp_ind in enumerate(prox_table):
        where_th_ind = np.where(ProxTablecopy == exp_ind)[0]
        if len(where_th_ind) > 1:
            for indy in where_th_ind:
                ProxTablecopy[indy] = -prox_table[indy]
            closest = np.argmin(allresidues[where_th_ind])
            ProxTablecopy[where_th_ind[closest]] = -ProxTablecopy[where_th_ind[closest]]
    
    singleindices = []
    refine_indexed_spots = {}
    # loop over close exp. spots
    for k in range(len(List_Exp_spot_close)):
        exp_index = List_Exp_spot_close[k]
        if not singleindices.count(exp_index):
            singleindices.append(exp_index)
            theo_index = np.where(ProxTablecopy == exp_index)[0]
            if (len(theo_index) == 1):  # only one theo spot close to the current exp. spot
                refine_indexed_spots[exp_index] = [exp_index, theo_index, Miller_Exp_spot[k]]
            else:  # recent PATCH:
                closest_theo_ind = np.argmin(allresidues[theo_index])
                if allresidues[theo_index][closest_theo_ind] < angtol:
                    refine_indexed_spots[exp_index] = [exp_index, theo_index[closest_theo_ind], Miller_Exp_spot[k]]       
    listofpairs = []
    theo_index = []
    linkResidues = []        
    selectedAbsoluteSpotIndices = np.arange(len(data_theta))
    for val in list(refine_indexed_spots.values()):
        if val[2] is not None:
            localspotindex = val[0]
            if not isinstance(val[1], (list, np.ndarray)):
                closetheoindex = val[1]
            else:
                closetheoindex = val[1][0]
            absolute_spot_index = selectedAbsoluteSpotIndices[localspotindex]
            listofpairs.append(absolute_spot_index)  # Exp, Theo,  where -1 for specifying that it came from automatic linking
            theo_index.append(closetheoindex)
            linkResidues.append(allresidues[closetheoindex])
    return listofpairs, linkResidues, theo_index

def calculate_strains_fromUB(s_tth, s_chi, UBmat, material_, input_params, 
                             detectorparameters, dict_dp, spots, B_matrix):
    # starting B0matrix corresponding to the unit cell   -----
    B0matrix = np.copy(B_matrix)
    latticeparams = dictLT.dict_Materials[material_][1]
    ## Included simple multi level refinement of strains
    init_residues = -0.1
    final_residues = -0.1
    
    if input_params["mat"] == 1:
        straintolerance = input_params["tolerancestrain"]
    elif input_params["mat"] == 2:
        straintolerance = input_params["tolerancestrain1"]
    
    devstrain, deviatoricstrain_sampleframe = np.zeros((3,3)), np.zeros((3,3))
    for ijk, AngTol in enumerate(straintolerance):
        #### Spots in first match (no refining, just simple auto links to filter spots)        
        grain = CP.Prepare_Grain(material_, UBmat, dictmaterials=dictLT.dict_Materials)
        Twicetheta, Chi, Miller_ind, posx, posy, _ = LT.SimulateLaue(grain,
                                                                 input_params["emin"], 
                                                                 input_params["emax"], 
                                                                 detectorparameters,
                                                                 kf_direction=dict_dp['kf_direction'],
                                                                 removeharmonics=1,
                                                                 pixelsize=dict_dp['pixelsize'],
                                                                 dim=dict_dp['dim'],
                                                                 ResolutionAngstrom=False,
                                                                 detectordiameter=dict_dp['detectordiameter'],
                                                                 dictmaterials=dictLT.dict_Materials)
        ## get proximity for exp and theo spots
        linkedspots_link, linkExpMiller_link, \
            linkResidues_link = getProximityv0(np.array([Twicetheta, Chi]),  # warning array(2theta, chi)
                                                                                s_tth/2.0, s_chi, Miller_ind,  # warning theta, chi for exp
                                                                                angtol=float(AngTol))
        
        if len(linkedspots_link) < 8:
            return np.zeros((3,3)), np.zeros((3,3)), init_residues, final_residues, UBmat
        
        linkedspots_fit = linkedspots_link
        linkExpMiller_fit = linkExpMiller_link
        
        arraycouples = np.array(linkedspots_fit)
        exp_indices = np.array(arraycouples[:, 0], dtype=np.int)
        sim_indices = np.array(arraycouples[:, 1], dtype=np.int)
    
        nb_pairs = len(exp_indices)
        Data_Q = np.array(linkExpMiller_fit)[:, 1:]
        sim_indices = np.arange(nb_pairs)  # for fitting function this must be an arange...
    
        pixX = np.take(dict_dp['peakX'], exp_indices)
        pixY = np.take(dict_dp['peakY'], exp_indices)
        weights = None #np.take(dict_dp['intensity'], exp_indices)
        
        starting_orientmatrix = np.copy(UBmat)
    
        results = None
        # ----------------------------------
        #  refinement model
        # ----------------------------------
        # -------------------------------------------------------
        allparameters = np.array(detectorparameters + [1, 1, 0, 0, 0] + [0, 0, 0])
        # strain & orient
        initial_values = np.array([1.0, 1.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0])
        arr_indexvaryingparameters = np.arange(5, 13)
    
        residues, deltamat, newmatrix = FitO.error_function_on_demand_strain(
                                                                            initial_values,
                                                                            Data_Q,
                                                                            allparameters,
                                                                            arr_indexvaryingparameters,
                                                                            sim_indices,
                                                                            pixX,
                                                                            pixY,
                                                                            initrot=starting_orientmatrix,
                                                                            Bmat=B0matrix,
                                                                            pureRotation=0,
                                                                            verbose=1,
                                                                            pixelsize=dict_dp['pixelsize'],
                                                                            dim=dict_dp['dim'],
                                                                            weights=weights,
                                                                            kf_direction=dict_dp['kf_direction'])
        init_mean_residues = np.copy(np.mean(residues))
        
        if ijk == 0:
            init_residues = np.copy(init_mean_residues)
        
        results = FitO.fit_on_demand_strain(initial_values,
                                                Data_Q,
                                                allparameters,
                                                FitO.error_function_on_demand_strain,
                                                arr_indexvaryingparameters,
                                                sim_indices,
                                                pixX,
                                                pixY,
                                                initrot=starting_orientmatrix,
                                                Bmat=B0matrix,
                                                pixelsize=dict_dp['pixelsize'],
                                                dim=dict_dp['dim'],
                                                verbose=0,
                                                weights=weights,
                                                kf_direction=dict_dp['kf_direction'])
    
        if results is None:
            return np.zeros((3,3)), np.zeros((3,3)), init_residues, final_residues, UBmat
    
        residues, deltamat, newmatrix = FitO.error_function_on_demand_strain(
                                                                            results,
                                                                            Data_Q,
                                                                            allparameters,
                                                                            arr_indexvaryingparameters,
                                                                            sim_indices,
                                                                            pixX,
                                                                            pixY,
                                                                            initrot=starting_orientmatrix,
                                                                            Bmat=B0matrix,
                                                                            pureRotation=0,
                                                                            verbose=1,
                                                                            pixelsize=dict_dp['pixelsize'],
                                                                            dim=dict_dp['dim'],
                                                                            weights=weights,
                                                                            kf_direction=dict_dp['kf_direction'])
        # if np.mean(residues) > final_residues:
        #     return devstrain, deviatoricstrain_sampleframe, init_residues, final_residues, UBmat
        final_mean_residues = np.copy(np.mean(residues))
        final_residues = np.copy(final_mean_residues)
        # building B mat
        # param_strain_sol = results
        # varyingstrain = np.array([[1.0, param_strain_sol[2], param_strain_sol[3]],
        #                                 [0, param_strain_sol[0], param_strain_sol[4]],
        #                                 [0, 0, param_strain_sol[1]]])
        # newUmat = np.dot(deltamat, starting_orientmatrix)
        # newUBmat = np.dot(newUmat, varyingstrain)
        newUBmat = np.copy(newmatrix) 
        # Bstar_s = np.dot(newUBmat, B0matrix)
        # ---------------------------------------------------------------
        # postprocessing of unit cell orientation and strain refinement
        # ---------------------------------------------------------------
        UBmat = np.copy(newmatrix) 
        (devstrain, lattice_parameter_direct_strain) = CP.compute_deviatoricstrain(newUBmat, B0matrix, latticeparams)
        # overwrite and rescale possibly lattice lengthes
        # constantlength = "a"
        # lattice_parameter_direct_strain = CP.computeLatticeParameters_from_UB(newUBmat, material_, constantlength, dictmaterials=dictLT.dict_Materials)
        # print(lattice_parameter_direct_strain)
        deviatoricstrain_sampleframe = CP.strain_from_crystal_to_sample_frame2(devstrain, newUBmat)
        # in % already
        devstrain = np.round(devstrain * 100, decimals=3)
        deviatoricstrain_sampleframe = np.round(deviatoricstrain_sampleframe * 100, decimals=3)
    return devstrain, deviatoricstrain_sampleframe, init_residues, final_residues, UBmat #initial_orientmatrix #UBmat

def getProximityv0(TwicethetaChi, data_theta, data_chi, data_hkl, angtol=0.5):
    # theo simul data
    theodata = np.array([TwicethetaChi[0] / 2.0, TwicethetaChi[1]]).T
    # exp data
    sorted_data = np.array([data_theta, data_chi]).T
    table_dist = GT.calculdist_from_thetachi(sorted_data, theodata)

    prox_table = np.argmin(table_dist, axis=1)
    allresidues = np.amin(table_dist, axis=1)
    very_close_ind = np.where(allresidues < angtol)[0]
    List_Exp_spot_close = []
    Miller_Exp_spot = []
    if len(very_close_ind) > 0:
        for theospot_ind in very_close_ind:  # loop over theo spots index
            List_Exp_spot_close.append(prox_table[theospot_ind])
            Miller_Exp_spot.append(data_hkl[theospot_ind])
    else:
        return [],[],[]
    # removing exp spot which appears many times(close to several simulated spots of one grain)--------------
    arrayLESC = np.array(List_Exp_spot_close, dtype=float)
    sorted_LESC = np.sort(arrayLESC)
    diff_index = sorted_LESC - np.array(list(sorted_LESC[1:]) + [sorted_LESC[0]])
    toremoveindex = np.where(diff_index == 0)[0]
    if len(toremoveindex) > 0:
        # index of exp spot in arrayLESC that are duplicated
        ambiguous_exp_ind = GT.find_closest(np.array(sorted_LESC[toremoveindex], dtype=float), arrayLESC, 0.1)[1]
        for ind in ambiguous_exp_ind:
            Miller_Exp_spot[ind] = None
    
    ProxTablecopy = np.copy(prox_table)

    for theo_ind, exp_ind in enumerate(prox_table):
        where_th_ind = np.where(ProxTablecopy == exp_ind)[0]
        if len(where_th_ind) > 1:
            for indy in where_th_ind:
                ProxTablecopy[indy] = -prox_table[indy]
            closest = np.argmin(allresidues[where_th_ind])
            ProxTablecopy[where_th_ind[closest]] = -ProxTablecopy[where_th_ind[closest]]
    
    singleindices = []
    refine_indexed_spots = {}
    # loop over close exp. spots
    for k in range(len(List_Exp_spot_close)):
        exp_index = List_Exp_spot_close[k]
        if not singleindices.count(exp_index):
            singleindices.append(exp_index)
            theo_index = np.where(ProxTablecopy == exp_index)[0]
            if (len(theo_index) == 1):  # only one theo spot close to the current exp. spot
                refine_indexed_spots[exp_index] = [exp_index, theo_index, Miller_Exp_spot[k]]
            else:  # recent PATCH:
                closest_theo_ind = np.argmin(allresidues[theo_index])
                if allresidues[theo_index][closest_theo_ind] < angtol:
                    refine_indexed_spots[exp_index] = [exp_index, theo_index[closest_theo_ind], Miller_Exp_spot[k]]
    
    listofpairs = []
    linkExpMiller = []
    linkResidues = []
    
    selectedAbsoluteSpotIndices = np.arange(len(data_theta))
    for val in list(refine_indexed_spots.values()):
        if val[2] is not None:
            localspotindex = val[0]
            if not isinstance(val[1], (list, np.ndarray)):
                closetheoindex = val[1]
            else:
                closetheoindex = val[1][0]
            absolute_spot_index = selectedAbsoluteSpotIndices[localspotindex]
            listofpairs.append([absolute_spot_index, closetheoindex])  # Exp, Theo,  where -1 for specifying that it came from automatic linking
            linkExpMiller.append([float(absolute_spot_index)] + [float(elem) for elem in val[2]])  # float(val) for further handling as floats array
            linkResidues.append([absolute_spot_index, closetheoindex, allresidues[closetheoindex]])

    linkedspots_link = np.array(listofpairs)
    linkExpMiller_link = linkExpMiller
    linkResidues_link = linkResidues
    return linkedspots_link, linkExpMiller_link, linkResidues_link

def get_ipf_colour(orientation_matrix1, axis=np.array([0., 0., 1.]), symmetry=None, symm_operator=None):
    """Compute the IPF (inverse pole figure) colour for this orientation.
    Given a particular axis expressed in the laboratory coordinate system,
    one can compute the so called IPF colour based on that direction
    expressed in the crystal coordinate system as :math:`[x_c,y_c,z_c]`.
    There is only one tuple (u,v,w) such that:
    .. math::
      [x_c,y_c,z_c]=u.[0,0,1]+v.[0,1,1]+w.[1,1,1]
    and it is used to assign the RGB colour.
    :param ndarray axis: the direction to use to compute the IPF colour.
    :param Symmetry symmetry: the symmetry operator to use.
    :return tuple: a tuple contining the RGB values.
    """
    if not np.all(orientation_matrix1==0):
        orientation_matrix = orientation_matrix1
    else:
        return 0,0,0
    # ## rotate orientation by 40degrees to bring in Sample RF
    omega = np.deg2rad(-40.0)
    # rotation de -omega autour de l'axe x (or Y?) pour repasser dans Rsample
    cw = np.cos(omega)
    sw = np.sin(omega)
    mat_from_lab_to_sample_frame = np.array([[cw, 0.0, sw], [0.0, 1.0, 0.0], [-sw, 0, cw]])
    orientation_matrix = np.dot(mat_from_lab_to_sample_frame.T, orientation_matrix)
    if np.linalg.det(orientation_matrix) < 0:
        orientation_matrix = -orientation_matrix
    axis /= np.linalg.norm(axis)
    
    # rgb = get_field_color(orientation_matrix, axis, symmetry=symmetry, syms=syms)            
    # return rgb

    Vc = np.dot(orientation_matrix, axis)
    # get the symmetry operators
    syms = np.array(symm_operator) #symmetry.symmetry_operators()
    syms = np.concatenate((syms, -syms))
    syms = np.unique(syms, axis=0)
    
    if symmetry == symmetry.cubic:
        rgb = get_field_color(orientation_matrix, axis, symmetry, syms)            
        return rgb
        # angleR = 45 - Vc_chi  # red color proportional to (45 - chi)
        # minAngleR = 0
        # maxAngleR = 45
        # angleB = Vc_phi  # blue color proportional to phi
        # minAngleB = 0
        # maxAngleB = 45
    elif symmetry == symmetry.hexagonal:
        Vc_syms = np.dot(syms, Vc)
        # phi: rotation around 001 axis, from 100 axis to Vc vector, projected on (100,010) plane
        Vc_phi = np.arctan2(Vc_syms[:, 1], Vc_syms[:, 0]) * 180 / np.pi
        # chi: rotation around 010 axis, from 001 axis to Vc vector, projected on (100,001) plane
        # Vc_chi = np.arctan2(Vc_syms[:, 0], Vc_syms[:, 2]) * 180 / np.pi
        # psi : angle from 001 axis to Vc vector
        Vc_psi = np.arccos(Vc_syms[:, 2]) * 180 / np.pi
        
        angleR = 90 - Vc_psi  # red color proportional to (90 - psi)
        minAngleR = 0
        maxAngleR = 90
        angleB = Vc_phi  # blue color proportional to phi
        minAngleB = 0
        maxAngleB = 30
    else:
        rgb = get_field_color(orientation_matrix, axis, symmetry, syms)            
        return rgb
    # find the axis lying in the fundamental zone
    fz_list = ((angleR >= minAngleR) & (angleR < maxAngleR) &
                (angleB >= minAngleB) & (angleB < maxAngleB)).tolist()
    if not fz_list.count(True) == 1:
        print("funda problem")
        rgb = get_field_color(orientation_matrix, axis, symmetry, syms)            
        return rgb
    i_SST = fz_list.index(True)
    r = angleR[i_SST] / maxAngleR
    g = (maxAngleR - angleR[i_SST]) / maxAngleR * (maxAngleB - angleB[i_SST]) / maxAngleB
    b = (maxAngleR - angleR[i_SST]) / maxAngleR * angleB[i_SST] / maxAngleB
    rgb = np.array([r, g, b])
    rgb = rgb / rgb.max()
    return rgb 

def get_field_color(orientation_matrix, axis=np.array([0., 0., 1.]), symmetry=None, syms=None):
    """Compute the IPF (inverse pole figure) colour for this orientation.
    Given a particular axis expressed in the laboratory coordinate system,
    one can compute the so called IPF colour based on that direction
    expressed in the crystal coordinate system as :math:`[x_c,y_c,z_c]`.
    There is only one tuple (u,v,w) such that:
    .. math::
      [x_c,y_c,z_c]=u.[0,0,1]+v.[0,1,1]+w.[1,1,1]
    and it is used to assign the RGB colour.
    :param ndarray axis: the direction to use to compute the IPF colour.
    :param Symmetry symmetry: the symmetry operator to use.
    :return tuple: a tuple contining the RGB values.
    """
    for sym in syms:
        Osym = np.dot(sym, orientation_matrix)
        Vc = np.dot(Osym, axis)
        if Vc[2] < 0:
            Vc *= -1.  # using the upward direction
        uvw = np.array([Vc[2] - Vc[1], Vc[1] - Vc[0], Vc[0]])
        uvw /= np.linalg.norm(uvw)
        uvw /= max(uvw)
        if (uvw[0] >= 0. and uvw[0] <= 1.0) and (uvw[1] >= 0. and uvw[1] <= 1.0) and (
                uvw[2] >= 0. and uvw[2] <= 1.0):
            break
    uvw = uvw / uvw.max()
    return uvw

In [3]:
# =============================================================================
# USER INPUT
# =============================================================================
use_om_user = False
nb_spots_consider = 100
residues_threshold=0.5
nb_spots_global_threshold=8
option_global = "v2"
CST_ENERGYKEV = 12.398

material_ = "Al2TiO5"
material1_ = material_
symmetry_name = "orthorhombic"
symmetry1_name = symmetry_name
SG = 63

ubmat = 2
use_previous_UBmatrix_name = False
strain_calculation = True

model_direc = r"C:\Users\purushot\Desktop\pattern_matching\Al2TiO5_jan2022"

detectorfile = r"C:\Users\purushot\Desktop\Al2TiO5_laue\old_results\data\VF_P1\Ge_VF_P1\Ge_VF_P1.det"
emin = 5
emax = 21

filenameDirec = r"C:\Users\purushot\Desktop\Al2TiO5_laue\old_results\data\VF_P1\VF_P1_R1"
experimental_prefix = "VF_P1_R1_"
lim_x, lim_y = 21, 21

tolerance = 0.6
tolerance1 = tolerance

tolerance_strain = [0.6,0.55,0.5,0.45,0.4,0.35,0.3,0.25,0.2,0.15]
tolerance_strain1 = tolerance_strain

intensity_threshold = 50 #75 800
boxsize = 10
fit_peaks_gaussian = 1
FitPixelDev = 18
NumberMaxofFits = 2000 ### Max peaks per LP
bkg_treatment = "A-B"

softmax_threshold_global = 0.80 # softmax_threshold
mr_threshold_global = 0.90 # match rate threshold
cap_matchrate = 0.01 * 100 ## any UB matrix providing MR less than this will be ignored
coeff = 0.10 ## should be same as cap_matchrate or no?
coeff_overlap = 0.10 ##10% spots overlap to avoid bad orientation detection
material0_limit = 100000
material1_limit = 100000
material_phase_always_present = "none"
# =============================================================================
# END OF USER INPUT
# =============================================================================

In [4]:
max_progress = lim_x * lim_y

#timermp1212.setInterval(500) ## check every second (update the list of files in folder)
#timermp1212.timeout.connect(update_data_mp)
#timermp1212.start()

if symmetry_name =="cubic":
    material0_lauegroup = "11"
elif symmetry_name =="monoclinic":
    material0_lauegroup = "2"
elif symmetry_name == "hexagonal":
    material0_lauegroup = "9"
elif symmetry_name == "orthorhombic":
    material0_lauegroup = "3"
elif symmetry_name == "tetragonal":
    material0_lauegroup = "5"
elif symmetry_name == "trigonal":
    material0_lauegroup = "7"
elif symmetry_name == "triclinic":
    material0_lauegroup = "1"

material1_lauegroup = material0_lauegroup

a, b, c, alpha, beta, gamma = dictLT.dict_Materials[material_][1]

if symmetry_name =="cubic":
    crystal = SGLattice(int(SG), a)
    symmetry = Symmetry.cubic
    lattice_material = Lattice.cubic(a)
elif symmetry_name =="monoclinic":
    crystal = SGLattice(int(SG),a, b, c, beta)
    symmetry = Symmetry.monoclinic
    lattice_material = Lattice.monoclinic(a, b, c, beta)
elif symmetry_name == "hexagonal":
    crystal = SGLattice(int(SG),a, c)
    symmetry = Symmetry.hexagonal
    lattice_material = Lattice.hexagonal(a, c)
elif symmetry_name == "orthorhombic":
    crystal = SGLattice(int(SG),a, b, c)
    symmetry = Symmetry.orthorhombic
    lattice_material = Lattice.orthorhombic(a, b, c)
elif symmetry_name == "tetragonal":
    crystal = SGLattice(int(SG),a, c)
    symmetry = Symmetry.tetragonal
    lattice_material = Lattice.tetragonal(a, c)
elif symmetry_name == "trigonal":
    crystal = SGLattice(int(SG),a, alpha)
    symmetry = Symmetry.trigonal
    lattice_material = Lattice.rhombohedral(a, alpha)
elif symmetry_name == "triclinic":
    crystal = SGLattice(int(SG),a, b, c, alpha, beta, gamma)
    symmetry = Symmetry.triclinic
    lattice_material = Lattice.triclinic(a, b, c, alpha, beta, gamma)

symmetry1 = None
lattice_material1 = None
crystal1 = None

lattice_, lattice1_ = lattice_material, lattice_material1

## load model related files and generate the model
json_file = open(model_direc+"//model_Al2TiO5.json", 'r')

classhkl = np.load(model_direc+"//MOD_grain_classhkl_angbin.npz")["arr_0"]
angbins = np.load(model_direc+"//MOD_grain_classhkl_angbin.npz")["arr_1"]

ind_mat = None
ind_mat1 = None  

load_weights = model_direc + "//model_Al2TiO5.h5"
wb = read_hdf5(load_weights)
temp_key = list(wb.keys())

# # load json and create model
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
print("Constructing model")
model.load_weights(load_weights)
print("Uploading weights to model")
print("All model files found and loaded")

ct = time.time()
now = datetime.datetime.fromtimestamp(ct)
c_time = now.strftime("%Y-%m-%d_%H-%M-%S")


hkl_all_class1 = None
with open(model_direc+"//classhkl_data_nonpickled_"+material_+".pickle", "rb") as input_file:
    hkl_all_class0 = cPickle.load(input_file)[0]


_file = open(detectorfile, "r")
text = _file.readlines()
_file.close()
# first line contains parameters
parameters = [float(elem) for elem in str(text[0]).split(",")]
detectorparameters = parameters[:5]
pixelsize = parameters[5]
dim1 = parameters[6]
dim2 = parameters[7]
# others are comments
comments = text[1:]
ccd_label_global = ""
for line in comments:
    if line.startswith("# CCDLabel"):
        ccd_label_global = line.split(":")[1].strip()
if ccd_label_global == "":
    print("CCD label cannot be read from the calibration file, setting it to latest detector sCMOS")
    ccd_label_global = "sCMOS"


col = [[] for i in range(int(ubmat))]
colx = [[] for i in range(int(ubmat))]
coly = [[] for i in range(int(ubmat))]
rotation_matrix = [[] for i in range(int(ubmat))]
strain_matrix = [[] for i in range(int(ubmat))]
strain_matrixs = [[] for i in range(int(ubmat))]
match_rate = [[] for i in range(int(ubmat))]
spots_len = [[] for i in range(int(ubmat))]
iR_pix = [[] for i in range(int(ubmat))]
fR_pix = [[] for i in range(int(ubmat))]
mat_global = [[] for i in range(int(ubmat))]
best_match = [[] for i in range(int(ubmat))]
spots1_global = [[] for i in range(int(ubmat))]
for i in range(int(ubmat)):
    col[i].append(np.zeros((lim_x*lim_y,3)))
    colx[i].append(np.zeros((lim_x*lim_y,3)))
    coly[i].append(np.zeros((lim_x*lim_y,3)))
    rotation_matrix[i].append(np.zeros((lim_x*lim_y,3,3)))
    strain_matrix[i].append(np.zeros((lim_x*lim_y,3,3)))
    strain_matrixs[i].append(np.zeros((lim_x*lim_y,3,3)))
    match_rate[i].append(np.zeros((lim_x*lim_y,1)))
    spots_len[i].append(np.zeros((lim_x*lim_y,1)))
    iR_pix[i].append(np.zeros((lim_x*lim_y,1)))
    fR_pix[i].append(np.zeros((lim_x*lim_y,1)))
    mat_global[i].append(np.zeros((lim_x*lim_y,1)))
    best_match[i].append([[] for jk in range(lim_x*lim_y)])
    spots1_global[i].append([[] for jk in range(lim_x*lim_y)])


if use_previous_UBmatrix_name:
    np.savez_compressed(model_direc+'//rotation_matrix_indexed_1.npz', rotation_matrix, mat_global, match_rate, 0.0)

# =============================================================================
#         ## Multi-processing routine
# =============================================================================        
## Number of files to generate
grid_files = np.zeros((lim_x,lim_y))
filenm = np.chararray((lim_x,lim_y), itemsize=1000)
grid_files = grid_files.ravel()
filenm = filenm.ravel()
count_global = lim_x * lim_y

format_file = dictLT.dict_CCD[ccd_label_global][7]

list_of_files = glob.glob(filenameDirec+'//'+experimental_prefix+'*.'+format_file)
## sort files
## TypeError: '<' not supported between instances of 'str' and 'int'
list_of_files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

if len(list_of_files) == count_global:
    for ii in range(len(list_of_files)):
        grid_files[ii] = ii
        filenm[ii] = list_of_files[ii]               
else:
    print("expected "+str(count_global)+" files based on the XY grid ("+str(lim_x)+","+str(lim_y)+") defined by user")
    print("But found "+str(len(list_of_files))+" files (either all data is not written yet or maybe XY grid definition is not proper)")
    digits = len(str(count_global))
    digits = max(digits,4)
    # Temp fix
    for ii in range(count_global):
        text = str(ii)
        if ii < 10000:
            string = text.zfill(4)
        else:
            string = text.zfill(5)
        file_name_temp = filenameDirec+'//'+experimental_prefix + string+'.'+format_file
        ## store it in a grid 
        filenm[ii] = file_name_temp

check = np.zeros((count_global,int(ubmat)))
# =============================================================================
blacklist = None

### Create a COR directory to be loaded in LaueTools
cor_file_directory = filenameDirec + "//" + experimental_prefix+"CORfiles"
if not os.path.exists(cor_file_directory):
    os.makedirs(cor_file_directory)

try_prevs = False
files_treated = []
mode_spotCycle = "graphmode_allHKL"

valu12 = [[filenm[ii].decode(), ii,
           rotation_matrix,
            strain_matrix,
            strain_matrixs,
            col,
            colx,
            coly,
            match_rate,
            spots_len, 
            iR_pix, 
            fR_pix,
            best_match,
            mat_global,
            check,
            detectorparameters,
            pixelsize,
            angbins,
            classhkl,
            hkl_all_class0,
            hkl_all_class1,
            emin,
            emax,
            material_,
            material1_,
            symmetry,
            symmetry1,   
            lim_x,
            lim_y,
            strain_calculation, 
            ind_mat, ind_mat1,
            model_direc, float(tolerance),
            float(tolerance1),
            int(ubmat), ccd_label_global, 
            None,
            float(intensity_threshold),
            int(boxsize),bkg_treatment,
            filenameDirec, 
            experimental_prefix,
            blacklist,
            None,
            files_treated,
            try_prevs, ## try previous is kept true, incase if its stuck in loop
            wb,
            temp_key,
            cor_file_directory,
            mode_spotCycle,
            softmax_threshold_global,
            mr_threshold_global,
            cap_matchrate,
            tolerance_strain,
            tolerance_strain1,
            NumberMaxofFits,
            fit_peaks_gaussian,
            FitPixelDev,
            coeff,
            coeff_overlap,
            material0_limit,
            material1_limit,
            use_previous_UBmatrix_name,
            material_phase_always_present,
            crystal,
            crystal1] for ii in range(count_global)]

#import threading
#timermp1212 = threading.Timer(1, update_data_mp)
#timermp1212.start()


#chunks = chunker_list(valu12, ncpu)
#chunks_mp = list(chunks)
#_inputs_queue = Queue()
#_outputs_queue = Queue()
#run_flag = multip.Value('I', True)
#_worker_processes = {}
#for i in range(ncpu):
#    _worker_processes[i]= Process(target=worker, args=(_inputs_queue, _outputs_queue, i+1, run_flag))
#for i in range(ncpu):
#    _worker_processes[i].start()   
#meta = {'t1':time.time()}
#for ijk in range(int(ncpu)):
#    _inputs_queue.put((chunks_mp[ijk], ncpu, meta))
    

Constructing model
Uploading weights to model
All model files found and loaded


In [ ]:
def new_MP_function(files, cnt, rotation_matrix, strain_matrix, strain_matrixs,\
                col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,mat_global,\
                check,detectorparameters,pixelsize,angbins,\
                classhkl, hkl_all_class0, hkl_all_class1, emin, emax,\
                material_, material1_, symmetry, symmetry1,lim_x,lim_y,\
                strain_calculation, ind_mat, ind_mat1,\
                model_direc, tolerance , tolerance1,\
                matricies, ccd_label,\
                filename_bkg,intensity_threshold,\
                boxsize,bkg_treatment,\
                filenameDirec, experimental_prefix,\
                blacklist_file, text_file, \
                files_treated,try_previous1,\
                wb, temp_key, cor_file_directory, mode_spotCycle1,\
                softmax_threshold_global123,mr_threshold_global123,\
                cap_matchrate123, tolerance_strain123, tolerance_strain1231,\
                NumberMaxofFits123,fit_peaks_gaussian_global123,\
                FitPixelDev_global123,coeff123,coeff_overlap,\
                material0_limit, material1_limit, use_previous_UBmatrix_name1,\
                    material_phase_always_present1, crystal, crystal1):
    print("working")            
    global total_progress
    total_progress += 1
    progress.value = total_progress/max_progress     

    try:
        strain_matrix12, strain_matrixs12, \
            rotation_matrix12, col12, \
                colx12, coly12,\
        match_rate12, mat_global12, cnt12,\
            files_treated12, spots_len12, \
                iR_pix12, fR_pix12, check12, best_match12 = predict_preprocessMultiProcess(files, cnt, 
                                                   rotation_matrix,strain_matrix,strain_matrixs,
                                                   col,colx,coly,match_rate,spots_len,iR_pix,fR_pix,best_match,
                                                   mat_global,
                                                   check,detectorparameters,pixelsize,angbins,
                                                   classhkl, hkl_all_class0, hkl_all_class1, emin, emax,
                                                   material_, material1_, symmetry, symmetry1,lim_x,lim_y,
                                                   strain_calculation, ind_mat, ind_mat1,
                                                   model_direc, tolerance, tolerance1,
                                                   matricies, ccd_label,
                                                   filename_bkg,intensity_threshold,
                                                   boxsize,bkg_treatment,
                                                   filenameDirec, experimental_prefix,
                                                   blacklist_file, text_file, 
                                                   files_treated,try_previous1,
                                                   wb, temp_key, cor_file_directory, mode_spotCycle1,
                                                   softmax_threshold_global123,mr_threshold_global123,
                                                   cap_matchrate123, tolerance_strain123,
                                                   tolerance_strain1231,NumberMaxofFits123,
                                                   fit_peaks_gaussian_global123,
                                                   FitPixelDev_global123, coeff123,coeff_overlap,
                                                   material0_limit,material1_limit,
                                                   use_previous_UBmatrix_name1,
                                                   material_phase_always_present1,
                                                   crystal, crystal1)
        return True, strain_matrix12, strain_matrixs12, rotation_matrix12, col12, \
                     colx12, coly12, match_rate12, mat_global12, cnt12, meta, \
                     files_treated12, spots_len12, iR_pix12, fR_pix12, best_match12, check12
    except Exception as e:
        return False, np.zeros((3,3))


if __name__ == "__main__":

    display(progress)
    
    import multiprocessing 
    
    with multiprocessing.Pool(ncpu) as pool:
        results = [pool.apply_async(new_MP_function, p) for p in valu12]
        for r in results:
            r_message_mpdata = r.get()
            if r_message_mpdata[0] == False:
                continue

            strain_matrix_mpdata, strain_matrixs_mpdata, rotation_matrix_mpdata, col_mpdata, \
                             colx_mpdata, coly_mpdata, match_rate_mpdata, mat_global_mpdata, \
                                 cnt_mpdata, meta_mpdata, files_treated_mpdata, spots_len_mpdata, \
                                     iR_pixel_mpdata, fR_pixel_mpdata, best_match_mpdata, check_mpdata = r_message_mpdata

            for i_mpdata in files_treated_mpdata:
                files_treated.append(i_mpdata)

            for intmat_mpdata in range(int(ubmat)):
                check[cnt_mpdata,intmat_mpdata] = check_mpdata[cnt_mpdata,intmat_mpdata]
                mat_global[intmat_mpdata][0][cnt_mpdata] = mat_global_mpdata[intmat_mpdata][0][cnt_mpdata]
                strain_matrix[intmat_mpdata][0][cnt_mpdata,:,:] = strain_matrix_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                strain_matrixs[intmat_mpdata][0][cnt_mpdata,:,:] = strain_matrixs_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                rotation_matrix[intmat_mpdata][0][cnt_mpdata,:,:] = rotation_matrix_mpdata[intmat_mpdata][0][cnt_mpdata,:,:]
                col[intmat_mpdata][0][cnt_mpdata,:] = col_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                colx[intmat_mpdata][0][cnt_mpdata,:] = colx_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                coly[intmat_mpdata][0][cnt_mpdata,:] = coly_mpdata[intmat_mpdata][0][cnt_mpdata,:]
                match_rate[intmat_mpdata][0][cnt_mpdata] = match_rate_mpdata[intmat_mpdata][0][cnt_mpdata]
                spots_len[intmat_mpdata][0][cnt_mpdata] = spots_len_mpdata[intmat_mpdata][0][cnt_mpdata]
                iR_pix[intmat_mpdata][0][cnt_mpdata] = iR_pixel_mpdata[intmat_mpdata][0][cnt_mpdata]
                fR_pix[intmat_mpdata][0][cnt_mpdata] = fR_pixel_mpdata[intmat_mpdata][0][cnt_mpdata]
                best_match[intmat_mpdata][0][cnt_mpdata] = best_match_mpdata[intmat_mpdata][0][cnt_mpdata]

    print("Launched all processes")  

FloatProgress(value=0.0, max=1.0)